## `Unspezifische Ereignisse`

### `Funktionen höherer Ordnung`

`Dieses Jupyter Notebook demonstriert praktisch den Umgang mit Funktionen höherer Ordnung in dritten Programmkomponenten.`

`Die exemplarische dritte Programmkomponente ist hier ein Übersetzer von Ereignisse in MIDI Dateien.` 

In [2]:
from mutwo import core_events
from mutwo import midi_converters
from mutwo import music_parameters

`Nehmen wir an wir formalisieren eine einfache Choreographie für eine Person. Die Choreographie besteht nur aus Kopfbewegungen. Es gibt vier unterschiedliche Bewegungsrichtungen (links, rechts, oben, unten). Zur Vereinfachung wird für die Richtung keine eigene Parameterklasse implementiert, sondern die Bewegungen werden direkt als Zeichenketten dargestellt.`

`Zu der Choreographie möchten wir dann eine Klangdatei erzeugen. Die Klangdatei soll je nach Bewegungsrichtung eine andere Tonhöhe spielen.`

`Zuerst wird eine eigene Ereignisklasse definiert, die die Kopfbewegung repräsentiert. Die Ereignisklasse erbt von der Ereignisbasisklasse *SimpleEvent*.`

In [3]:
# Deklaration aller Bewegungsrichtungen
LEFT, RIGHT, UP, DOWN = "left", "right", "up", "down"
# Einfache globale Zuweisung von Richtung zu Tonhöhe
DIRECTION_TO_PITCH = {
    LEFT: music_parameters.JustIntonationPitch('3/2'),
    RIGHT: music_parameters.JustIntonationPitch('4/3'),
    UP: music_parameters.JustIntonationPitch('5/4'),
    DOWN: music_parameters.JustIntonationPitch('4/5')
}

class HeadMovement(core_events.SimpleEvent):
    # Wir benutzen eine konstante Dynamik.
    volume = music_parameters.WesternVolume('mp')
    
    def __init__(self, direction, duration):
        self.direction = direction
        
        # Das ist Python spezifischer Syntax, um dem Konstruktor
        # der Basisklasse das Dauerargument zu übermitteln.
        super().__init__(duration)
    
    @property
    def pitch(self):
        return DIRECTION_TO_PITCH[self.direction]

`Als nächstens definieren wir unsere einfache Choreographie, die aus vier sequentiellen Bewegungen besteht.`

In [4]:
choreography = core_events.SequentialEvent([
    HeadMovement(LEFT, 1),
    HeadMovement(UP, 0.5),
    HeadMovement(RIGHT, 2),
    HeadMovement(DOWN, 3)
])

`Jetzt möchten wir die Choreographie in eine MIDI Datei übersetzen. In der [API Dokumentation der Klasse](https://mutwo.readthedocs.io/en/latest/api/mutwo.midi_converters.html#mutwo.midi_converters.EventToMidiFile) können wir nachlesen, dass diese unter anderem das Argument *simple_event_to_pitch_list* hat. Wir können auch nachlesen, dass dieses Argument standardisiert davon ausgeht, dass ein SimpleEvent ein Attribut namens *pitch_list* hat, was *HeadMovement* aber nicht hat. Deswegen müssen wir das Argument überschreiben, sodass es unser *pitch* Attribut finden kann.`

`Wir werden es hier mit einer anonymen Lambdafunktion überschreiben, die die Tonhöhe in einer Liste zurückgibt.`

In [6]:
event_to_midi_file = midi_converters.EventToMidiFile(
    simple_event_to_pitch_list=lambda simple_event: [
        getattr(simple_event, 'pitch')
    ]
)

`Zum Schluss kann die MIDI Datei erzeugt werden.`

In [7]:
midi_file = event_to_midi_file(choreography, 'choreography.mid')

[[JustIntonationPitch('3/2')], WesternVolume(mp), ()]
(Message('pitchwheel', channel=0, pitch=80, time=0), Message('note_on', channel=0, note=76, velocity=57, time=0), Message('note_off', channel=0, note=76, velocity=57, time=480))
[[JustIntonationPitch('5/4')], WesternVolume(mp), ()]
(Message('pitchwheel', channel=1, pitch=-561, time=479), Message('note_on', channel=1, note=73, velocity=57, time=480), Message('note_off', channel=1, note=73, velocity=57, time=720))
[[JustIntonationPitch('4/3')], WesternVolume(mp), ()]
(Message('pitchwheel', channel=2, pitch=-80, time=719), Message('note_on', channel=2, note=74, velocity=57, time=720), Message('note_off', channel=2, note=74, velocity=57, time=1680))
[[JustIntonationPitch('4/5')], WesternVolume(mp), ()]
(Message('pitchwheel', channel=3, pitch=561, time=1679), Message('note_on', channel=3, note=65, velocity=57, time=1680), Message('note_off', channel=3, note=65, velocity=57, time=3120))


/home/levinericzimmermann/Texts/21BAmutwo/jupyter/.venv/lib/python3.9/site-packages/mutwo/core_events/envelopes.py:382: InvalidAverageValueStartAndEndWarning: Average value for range where start == end is equal to envelope.value_at(start)!
  warnings.warn(core_utilities.InvalidAverageValueStartAndEndWarning())
